# PROYECTO INTEGRADOR: STEAM
## Dataset - australian_user_reviews.json

## 1) Ingesta de los datos

Se descomprimen cada uno de los archivos previo a importarlos con pandas.

In [1]:
#Importamos las librerías que utilizaremos a lo largo de nuestro proyecto
import pandas as pd
from pandas import json_normalize
from textblob import TextBlob
import json
import ast 

Importamos la data del archivo australian_user_reviews.json.

In [2]:
# Se carga en un dataframe las reviews de los usuarios.
user_reviews = []
with open('Data/australian_user_reviews.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        user_reviews.append(ast.literal_eval(line))

# Crear el DataFrame anidado
df_reviews_anidado = pd.DataFrame(user_reviews)


In [3]:
#Observamos el dataframe
df_reviews_anidado.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [4]:
#Imprimimos las dimensiones del dataframe
df_reviews_anidado.shape

(25799, 3)

## 2) Tratamiento de los datos

In [5]:
#Observamos la información contenida en un registro de la columna reviews
df_reviews_anidado['reviews'][0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

Observamos que en la columna reviews, cada registro está conformado por varios json. Por lo tanto, procederemos a desanidar la información contenida en la columna reviews.

In [6]:
# Desagregamos la columna reviews
df_reviews_anidado = df_reviews_anidado.explode('reviews')

In [7]:
#Observamos que se haya realizado la desagregación correctamente
df_reviews_anidado['reviews']

0        {'funny': '', 'posted': 'Posted November 5, 20...
0        {'funny': '', 'posted': 'Posted July 15, 2011....
0        {'funny': '', 'posted': 'Posted April 21, 2011...
1        {'funny': '', 'posted': 'Posted June 24, 2014....
1        {'funny': '', 'posted': 'Posted September 8, 2...
                               ...                        
25797    {'funny': '', 'posted': 'Posted July 10.', 'la...
25797    {'funny': '', 'posted': 'Posted July 8.', 'las...
25798    {'funny': '1 person found this review funny', ...
25798    {'funny': '', 'posted': 'Posted July 20.', 'la...
25798    {'funny': '', 'posted': 'Posted July 2.', 'las...
Name: reviews, Length: 59333, dtype: object

In [8]:
#Reseteamos los índices
df_reviews_anidado.reset_index(drop=True, inplace=True)

Vamos a trabajar con los registros nulos.

In [9]:
#Verificamos los nulos
df_reviews_anidado.isnull().sum()

user_id      0
user_url     0
reviews     28
dtype: int64

Decido eliminar los nulos de la columna reviews, porque al ser precisamente el dataframe de reviews, es la información más importante que quiero mantener. Además, es necesario eliminarlos para un correcto funcionamiento de la API en las funciones que dependan de la columna creada en el análisis de sentimiento.

In [10]:
#Elimino los valores nulos de la columna reviews
df_reviews_anidado = df_reviews_anidado.dropna(subset=['reviews'])

In [11]:
df_reviews_anidado.isnull().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

In [12]:
#Revisamos el dataframe
df_reviews_anidado.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."


Ahora vamos a desanidar los json contenidos en la columna review.

In [13]:
#Aplanamos los archivos json de la columna reviews
df_reviews = json_normalize(df_reviews_anidado['reviews'])

In [14]:
#Revisamos como nos quedó el dataframe desanidado
df_reviews.head()

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [15]:
#Concateno el dataframe al que le apliqué el explode y el dataframe al que le apliqué el json_normalize
df_reviews=pd.concat([df_reviews_anidado,df_reviews], axis=1)

Vamos a revisar los valores nulos del dataframe para determinar si hay valores que se pueden eliminar.

In [16]:
#Revisamos los valores nulos
df_reviews.isnull().sum()

user_id        28
user_url       28
reviews        28
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64

Como en las funciones, se nos solicita algún requerimiento ya sea del user_id o de las reviews (análisis de sentimiento), procederemos a  eliminar los 28 registros a partir de user_id.

In [17]:
#Eliminamos los valores donde user_id es nulo
df_reviews=df_reviews.dropna(subset='user_id')

In [18]:
#Revisamos los valores nulos nuevamente
df_reviews.isnull().sum()

user_id         0
user_url        0
reviews         0
funny          28
posted         28
last_edited    28
item_id        28
helpful        28
recommend      28
review         28
dtype: int64

Comprobamos que los 28 registros de user_id eliminados, no corresponden con los 28 registros nulos de review, por lo que ahora pasamos a eliminar los registros donde la review es nula.

In [19]:
#Eliminamos también valores donde la columna 'review' es vacío.
df_reviews=df_reviews.dropna(subset='review')

In [20]:
#Chequeamos que ya no haya valores nulos
df_reviews.isnull().sum()

user_id        0
user_url       0
reviews        0
funny          0
posted         0
last_edited    0
item_id        0
helpful        0
recommend      0
review         0
dtype: int64

In [21]:
#Imprimimos las dimensiones del dataframe
df_reviews.shape

(59277, 10)

In [22]:
#Revisamos nuestro dataframe
df_reviews.head()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


Decidimos eliminar las columnas que no vamos a utilizar para las funciones o para el EDA.

In [23]:
#Eliminamos las columnas que no utilizaremos
df_reviews=df_reviews.drop(columns=['user_url','reviews','funny','last_edited','helpful'])

In [24]:
#Verificamos la correcta eliminación de las columnas
df_reviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
4,js41637,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...


Queremos que de la información de la columna posted podamos extraer el año (year_review) que es un dato que se nos solicita en las funciones de recomendaciones. Para ello, separamos el contenido de la columna posted en columnas y procedemos a eliminar la palabra posted y el punto final.

In [25]:
#Reemplazamos la palabra posted por una cadena vacía
df_reviews['posted'] = df_reviews['posted'].str.replace('Posted ', '')

In [26]:
#Reemplazamos el punto final por una cadena vacía
df_reviews['posted'] = df_reviews['posted'].str.replace('.', '')

In [27]:
#Verificamos la realización de los dos pasos anteriores.
df_reviews.head()

,user_id,posted,item_id,recommend,review
0,76561197970982479,"November 5, 2011",1250,True,Simple yet with great replayability. In my opi...
1,76561197970982479,"July 15, 2011",22200,True,It's unique and worth a playthrough.
2,76561197970982479,"April 21, 2011",43110,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,"June 24, 2014",251610,True,I know what you think when you see this title ...
4,js41637,"September 8, 2013",227300,True,For a simple (it's actually not all that simpl...


In [28]:
#Dividimos a partir de la coma la información en day_review y en year_review
df_reviews[['day_review', 'year_review']]=df_reviews['posted'].str.split(', ', expand=True)

In [29]:
#Revisamos la info de nuestro dataframe
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59277 entries, 0 to 59304
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59277 non-null  object
 1   posted       59277 non-null  object
 2   item_id      59277 non-null  object
 3   recommend    59277 non-null  object
 4   review       59277 non-null  object
 5   day_review   59277 non-null  object
 6   year_review  49162 non-null  object
dtypes: object(7)
memory usage: 3.6+ MB


Las recomendaciones que piden las funciones, son en función de year_review, por lo tanto decidimos eliminar los valores que son nulos en esta columna.

In [30]:
#Tomamos los datos de df_reviews que no son nulos. 
df_reviews = df_reviews[df_reviews['year_review'].notna()]

In [31]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49162 entries, 0 to 59276
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      49162 non-null  object
 1   posted       49162 non-null  object
 2   item_id      49162 non-null  object
 3   recommend    49162 non-null  object
 4   review       49162 non-null  object
 5   day_review   49162 non-null  object
 6   year_review  49162 non-null  object
dtypes: object(7)
memory usage: 3.0+ MB


In [32]:
#Chequeamos el tipo de dato de la columna year
df_reviews['year_review'].info()

<class 'pandas.core.series.Series'>
Index: 49162 entries, 0 to 59276
Series name: year_review
Non-Null Count  Dtype 
--------------  ----- 
49162 non-null  object
dtypes: object(1)
memory usage: 768.2+ KB


In [33]:
#Pasamos el dato de object a entero
df_reviews['year_review']=df_reviews['year_review'].astype(int)

In [34]:
#Verificamos que year_review ahora es un entero
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49162 entries, 0 to 59276
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      49162 non-null  object
 1   posted       49162 non-null  object
 2   item_id      49162 non-null  object
 3   recommend    49162 non-null  object
 4   review       49162 non-null  object
 5   day_review   49162 non-null  object
 6   year_review  49162 non-null  int32 
dtypes: int32(1), object(6)
memory usage: 2.8+ MB


In [35]:
#Verificamos los años en los que se hicieron los posted
df_reviews['year_review'].unique()

array([2011, 2014, 2013, 2015, 2012, 2010])

In [36]:
#Verificamos las dimensiones del dataframe
df_reviews.shape

(49162, 7)

Vamos a proceder a eliminar las columnas que no utilizaremos en las funciones: posted y day_review

In [37]:
#Eliminamos las columnas mencionadas
df_reviews=df_reviews.drop(columns=['posted','day_review'])

In [38]:
#Verificamos las dimensiones del dataframe luego de eliminar algunas columnas
df_reviews.shape

(49162, 5)

In [39]:
#Revisamos nuestro dataframe
df_reviews.head()

,user_id,item_id,recommend,review,year_review
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,251610,True,I know what you think when you see this title ...,2014
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013


#### 2.2.1) Sentiment analysis

Vamos a trabajar ahora con el dataframe df_reviews.<br>
Crearemos una columna llamada 'sentimeiento' aplicando un análisis de sentimiento con NLP tomando los siguientes valores:
- Debe tomar el valor '0' si es malo,
- Debe tomar el valor '1' si es neutral y
- Debe tomar el valor '2' si es positivo. <br>
De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

Para realizar el análisis de sentimiento se va a utilizar la librería textblob que es una librería de procesamiento del texto para Python que permite realizar tareas de Procesamiento del Lenguaje Natural.

Para trabajar nos vamos a centrar especificamente en la polaridad, que es un valor que va entra -1 y 1. Por lo tanto, si digo que la polaridad es negativa, el sentimiento es negativo, si es a cero es neutral (o ambiguo) y si mayor a 0, será positiva.

La consigna pide que los que estén vacíos en review se consideren para el sentiment analysis, sin embargo, como los vacíos no tienen valores en la columna recommend, se decide eliminarlos de todas maneras, para no tener nulos en las funciones que solicitan la recomendación. Además, los datos que no tienen información en year_review tampoco tienen información en review, por lo que se eliminan para simplificar las funciones en la API.

Definimos una función que al pasarle las reviews me identifique si son sentimientos positivos, neutros o negativos.

In [40]:
def analizar_sentimiento(texto):
    # Crear un objeto TextBlob 
    blob = TextBlob(texto)
    
    # Obtener el puntaje de sentimiento en un rango de -1 a 1
    sentimiento = blob.sentiment.polarity

    # Asignar valores numéricos según el análisis de sentimiento
    if sentimiento > 0:
        return 2  # Positivo
    elif sentimiento < 0:
        return 0  # Negativo
    else:
        return 1  # Neutral

In [41]:
#Aplicamos la función a la columna review del dataframe df_reviews.
df_reviews['sentimiento']=df_reviews['review'].apply(analizar_sentimiento)

In [42]:
#Chequeamos con los primeros 15 valores si la función tiene sentido a cómo se van clasificando los sentimientos.
df_reviews.head(15)

,user_id,item_id,recommend,review,year_review,sentimiento
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,2
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,2
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,2
3,js41637,251610,True,I know what you think when you see this title ...,2014,2
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013,0
5,js41637,239030,True,Very fun little game to play when your bored o...,2013,0
7,evcentric,370360,True,"""Run for fun? What the hell kind of fun is that?""",2015,2
8,evcentric,237930,True,"Elegant integration of gameplay, story, world ...",2014,2
9,evcentric,263360,True,"Random drops and random quests, with stat poin...",2014,0
10,evcentric,107200,True,Fun balance of tactics and strategy. Potentia...,2014,2


In [43]:
#Contamos para la columna sentimiento los valores
conteo_sentimiento = df_reviews['sentimiento'].value_counts()
conteo_sentimiento

sentimiento
2    28154
1    10698
0    10310
Name: count, dtype: int64

In [44]:
#Revisamos el tipo de dato de la columna sentimiento
df_reviews['sentimiento'].info()

<class 'pandas.core.series.Series'>
Index: 49162 entries, 0 to 59276
Series name: sentimiento
Non-Null Count  Dtype
--------------  -----
49162 non-null  int64
dtypes: int64(1)
memory usage: 768.2 KB


La columna sentimiento debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. Por lo tanto, procederemos a eliminar la columna review.

In [45]:
#Eliminamos la columna review
df_reviews=df_reviews.drop(columns=['review'])

In [46]:
df_reviews.head()

,user_id,item_id,recommend,year_review,sentimiento
0,76561197970982479,1250,True,2011,2
1,76561197970982479,22200,True,2011,2
2,76561197970982479,43110,True,2011,2
3,js41637,251610,True,2014,2
4,js41637,227300,True,2013,0


In [47]:
#Revisamos las dimensiones del dataframe
df_reviews.shape

(49162, 5)

In [48]:
#Guardamos en un csv
df_reviews.to_csv('Data/reviews.csv', index=False)